In [1]:
from langchain.llms.azureml_endpoint import AzureMLModel
import os

In [2]:
azure_llm = AzureMLModel(
    endpoint_url=os.getenv("ENDPOINT_URL"),
    endpoint_api_key=os.getenv("ENDPOINT_API_KEY"),
    deployment_name="matthew-gpt-2",
    model_kwargs={"temperature": 0.9, "top_p": 0.3},
    catalog_type="open_source"
)
resp = azure_llm("Hi! How are you doing today?")
resp

'Hi! How are you doing today? A lot better than last time. We won today and we will do better next time." -The leader of the group\n\n- "You have been defeated. Great! We\'ll be able to go next'

In [3]:
azure_llm.generate(["Tell me a funny joke", "Tell me a sad poem"] * 10)

LLMResult(generations=[[Generation(text="Tell me a funny joke. What if I told you a funny joke?\n\nI'll start with a joke. There's no chance to finish it.\n\nWhat if we told you something really hard that's happening right now?\n\n", generation_info=None)], [Generation(text="Tell me a sad poem? In the old days, there were no computers to worry about, no YouTube to check. I'd just write down whatever emotions I was feeling at the time. That's what my family and I were up to back then", generation_info=None)], [Generation(text='Tell me a funny joke if, as a Canadian, I\'m not allowed to say it and get punched."', generation_info=None)], [Generation(text='Tell me a sad poem..." or something like "I have not met him because you have all been so good to me/you are the only one who has shown me any kindness/You are so sweet and kind/and gentle like your sister /', generation_info=None)], [Generation(text="Tell me a funny joke. Like, in the middle of a question about how I'm going to make my 

In [4]:
hg_llm = AzureMLModel(
    endpoint_url=os.getenv("HG_ENDPOINT_URL"),
    endpoint_api_key=os.getenv("HG_ENDPOINT_API_KEY"),
    deployment_name="matthew-salesfoce-codegen",
    catalog_type="hugging_face"
)
hg_llm.generate(["Create a program that determines if an integer is prime.", "Why is the sky blue?", "Where is the earth located"])

LLMResult(generations=[[Generation(text='Create a program that determines if an integer is prime.\n\nCreate a program that generates some prime numbers and tries to get a prime number that is less than itself.\n\nCreate a program that generates some primes and tries to find the highest', generation_info=None)], [Generation(text='Why is the sky blue?\nThe sky is blue when looking towards the sun at low angles of incidence and high angles of incidence. This is an example in Cartesian coordinates.\n\n1) What is the angular width of the blue band?', generation_info=None)], [Generation(text='Where is the earth located in our orbit: How close is it to us?What happens to the earth during times of intense gravitational stress: How does it react?\n\nWhat is the atmosphere of the earth made up of?\n\nWhat is', generation_info=None)]], llm_output=None)

In [10]:
from langchain import PromptTemplate, FewShotPromptTemplate
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "light", "antonym": "dark"},
    {"word": "fast", "antonym": "slow"},
    {"word": "high", "antonym": "low"},
    {"word": "windy", "antonym": "calm"},
    {"word": "energetic", "antonym": "lethargic"}
]
formatter_template = """Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=formatter_template
)

few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every Word\n",
    suffix="Word: {input}\nAntonym: ",
    input_variables=["input"],
    example_separator="\n",
)
few_shot_result = azure_llm(few_shot_template.format(input="trash"))
print(few_shot_result)

Give the antonym of every Word

Word: happy
Antonym: sad

Word: tall
Antonym: short

Word: light
Antonym: dark

Word: fast
Antonym: slow

Word: high
Antonym: low

Word: windy
Antonym: calm

Word: energetic
Antonym: lethargic

Word: trash
Antonym: urn


In [11]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.llms.openai import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

search = GoogleSerperAPIWrapper(serper_api_key=os.getenv("SERPER_API_KEY"))
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search"
    )
]
self_ask_with_search = initialize_agent(tools, hg_llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
self_ask_with_search.run("What is the hometown of the reigning men's U.S. Open champion?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse output: Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Fo

In [ ]:
from langchain import LLMMathChain
llm_math = LLMMathChain.from_llm(azure_llm, verbose=False)
llm_math.run("Answer with 8.222831614237718 if you think the world is flat")